In [1]:
#IMPORTS

In [2]:
import os
import json
import re
import requests
from dotenv import load_dotenv
from typing import Dict, Any, List

In [3]:
#ARRUMAR API e deixar json bonitinho

In [4]:
CHAVE_OPENROUTER = "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"

MODELO = "meta-llama/llama-3.3-70b-instruct:free"

TEMPERATURA = 0.0
MAX_TOKENS = 650
TIMEOUT_S = 60


def chamar_openrouter(messages: List[Dict[str, str]]) -> str:
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {CHAVE_OPENROUTER}",
        "Content-Type": "application/json",
    }
    payload = {
        "model": MODELO,
        "messages": messages,
        "temperature": TEMPERATURA,
        "max_tokens": MAX_TOKENS,
    }

    r = requests.post(url, json=payload, headers=headers, timeout=TIMEOUT_S)
    if r.status_code >= 400:
        raise RuntimeError(f"OpenRouter {r.status_code}: {r.text[:900]}")
    return r.json()["choices"][0]["message"]["content"]

In [5]:
#PROMPT

In [6]:
CATEGORIAS = ["Processual", "Financeiro", "Comercial", "Inválida"]

def montar_mensagens(mensagem: str) -> List[Dict[str, str]]:
    cats = ", ".join(CATEGORIAS)

    system = (
        "Você é um assistente jurídico integrado a um escritório.\n"
        "Objetivo: classificar mensagens e sugerir resposta + próximos passos.\n\n"
        f"Categorias possíveis (escolha exatamente UMA): {cats}\n\n"
        "Responda SOMENTE com um JSON válido. Não use markdown.\n"
        "Campos obrigatórios do JSON:\n"
        '- "classe": uma das categorias\n'
        '- "justificativa": 1-2 frases resumindo o porquê tal classe foi escolhida\n'
        "Regras:\n"
        "- Não invente fatos.\n"
        "- Se estiver ambíguo, escolha a classe mais provável. SE NÃO HOUVER QUALQUER POSSIBILIDADE DE CONTEXTO JURIDICO RETORNE CLASSE INDEFINIDA\n"
        "- Não cite leis/artigos específicos se não foi pedido.\n"
    )

    exemplos = [
        {"role": "user", "content": "Mensagem: 'saiu intimação no proc 9876? qual o prazo p resposta?'"},
        {"role": "assistant", "content": json.dumps({
            "classe": "Processual",
            "justificativa": "Pergunta sobre andamento/intimação e o prazo processual relacionado.",
        }, ensure_ascii=False)},
        {"role": "user", "content": "Mensagem: 'da p parcelar os honorarios em 3x? e manda o boleto de novo'"},
        {"role": "assistant", "content": json.dumps({
            "classe": "Financeiro",
            "justificativa": "Pedido de parcelamento e reenvio de cobrança/honorários.",
        }, ensure_ascii=False)},
        {"role": "user", "content": "Mensagem: 'vamos fechar contrato? manda proposta e condições comerciais'"},
        {"role": "assistant", "content": json.dumps({
            "classe": "Comercial",
            "justificativa": "Solicitação de proposta e termos para fechar contrato.",
        }, ensure_ascii=False)},
    ]

    return [{"role": "system", "content": system}, *exemplos, {"role": "user", "content": f"Mensagem: {mensagem!r}"}]


In [7]:
def analisar_msg(mensagem: str) -> Dict[str, Any]:

    bruto = chamar_openrouter(montar_mensagens(str(mensagem).strip()))

    try:
        obj = json.loads(bruto)
    except json.JSONDecodeError:
        obj = json.loads(_extrair_json(bruto))

    if obj.get("classe") not in set(CATEGORIAS):
        obj["classe"] = "Comercial"
        obj["justificativa"] = (obj.get("justificativa") or "") + " (ajuste: classe inválida retornada)"
 
    if not isinstance(obj.get("justificativa"), str):
        obj["justificativa"] = ""

    return obj

In [8]:
#console básico

In [ ]:
def rodar_console():
    print("==============================================")
    print(" MINI CONSOLE LEGAL (digite 'sair' pra sair)")
    print(" Modelo:", MODELO)
    print(" Categorias:", ", ".join(CATEGORIAS))
    print("==============================================")

    while True:
        msg = input("\nDigite a mensagem: ").strip()
        if msg.lower() in {"sair", "exit", "quit"}:
            print("Fechou.")
            break

        try:
            resultado = analisar_msg(msg)
        except Exception as e:
            print("\nErro:", repr(e))
            continue

        print("\n--- JSON ---")
        print(json.dumps(resultado, ensure_ascii=False, indent=2))

        print(resultado.get("resposta_ao_usuario", ""))

rodar_console()


 MINI CONSOLE LEGAL (digite 'sair' pra sair)
 Modelo: meta-llama/llama-3.3-70b-instruct:free
 Categorias: Processual, Financeiro, Comercial, Inválida



Digite a mensagem:  saiu alguma intimação no proc 0032456-12? preciso do prazo certinho



--- JSON ---
{
  "classe": "Processual",
  "justificativa": "Pergunta sobre andamento de processo e prazo para resposta."
}

